In [136]:
import numpy as np
import pandas as pd
import seaborn

movies = pd.read_csv("movie.csv", header=0, index_col="movie id")
ratings = pd.read_csv("ratings.csv", header=0)
user = pd.read_csv("user.csv", header=0, index_col="user id")

# Questions

### How do we view the first 5 rows from each dataset?

### How do we view the shape of each dataset?

### How do we view the data types of each column?

### How do we provide a statistical summary of each dataset?

### How many movies are there per genre?

### How many movies are categorized in more than one genre?

In [137]:
movies['num_genres'] = movies.loc[:,'Action':'Western'].sum(axis=1).sort_values(ascending = False)
movies[movies['num_genres'] > 1].shape[0]

849

### Which 25 movies have the highest average ratings?

In [138]:
ratings.head()

# By using the agg function we can compute multiple aggregations in one group by
mean_ratings = ratings.groupby('movie id').agg({'rating': ['mean', 'count']})

# Because we have done multiple aggregations on the same column we have created a multi-index column
# referring to it we need to use a tuple
mean_ratings = mean_ratings[mean_ratings[('rating', 'count')] > 100]

# Sort the values and grab the top 25
mean_ratings = mean_ratings.sort_values(by=('rating', 'mean'), ascending=False).head(25)
top25 = mean_ratings.index
mean_ratings

rating      
              mean count
movie id                
408       4.491071   112
318       4.466443   298
169       4.466102   118
483       4.456790   243
64        4.445230   283
603       4.387560   209
12        4.385768   267
50        4.358491   583
178       4.344000   125
134       4.292929   198
427       4.292237   219
357       4.291667   264
98        4.289744   390
480       4.284916   179
127       4.283293   413
285       4.265432   162
272       4.262626   198
657       4.259542   131
474       4.252577   194
174       4.252381   420
479       4.251397   179
313       4.245714   350
511       4.231214   173
484       4.210145   138
172       4.204360   367

In [139]:
# We can perform an isin instead of a merge (has advantages if the input list is small, merging can take up 
# a lot of memory)
movies[movies.index.isin(top25)]

,movie title,release date,Action,Adventure,Animation,Childrens,Comedy,Crime,Documentary,Drama,...,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,num_genres
movie id,,,,,,,,,,,,,,,,,,,,,
12,"Usual Suspects, The",14-Aug-95,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,1,0,0,2
50,Star Wars,1-Jan-77,1,1,0,0,0,0,0,0,...,0,0,0,0,1,1,0,1,0,5
64,"Shawshank Redemption, The",1-Jan-94,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1
98,"Silence of the Lambs, The",1-Jan-91,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,1,0,0,2
127,"Godfather, The",1-Jan-72,1,0,0,0,0,1,0,1,...,0,0,0,0,0,0,0,0,0,3
134,Citizen Kane,1-Jan-41,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1
169,"Wrong Trousers, The",1-Jan-93,0,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,2
172,"Empire Strikes Back, The",1-Jan-80,1,1,0,0,0,0,0,1,...,0,0,0,0,1,1,0,1,0,6
174,Raiders of the Lost Ark,1-Jan-81,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2


### What is the gender distribution across the genres?

In [140]:
# You can specify the merge arguments to optimize runtime - left_index=True signifies to the function
# to use the left DataFrame's index to perform the merge (similarly, right_index=True for the right DataFrame)
gender_ratings = user[['age', 'gender']].merge(
    ratings[['user id', 'movie id', 'rating']], left_index=True, right_on='user id', how='inner'
)

In [141]:
# Now merge on the movies DataFrame
gender_movie_ratings = movies.merge(gender_ratings, left_index=True, right_on='movie id', how='inner')

# Group by gender and sum,  retrieve the genre columns
genre_gender = gender_movie_ratings.groupby("gender").sum().loc[:,'Action':'Western']

# Add row total of the DataFrame to get the total number of males and females who gave ratings
genre_gender['total'] = gender_movie_ratings['gender'].value_counts()
genre_gender

,Action,Adventure,Animation,Childrens,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,total
gender,,,,,,,,,,,,,,,,,,,
F,5442,3141,995,2232,8068,1794,187,11008,363,385,1197,1442,1314,5858,2629,5086,2189,371,25738
M,20147,10612,2610,4950,21764,6261,571,28887,989,1348,4120,3512,3931,13603,10101,16786,7209,1483,74252


In [142]:
# What percentage of each gender reviewed each genre?
# Axis = 0 specifies to perform the operation row-wise
(genre_gender.div(genre_gender.total, axis=0) * 100).T

gender,F,M
Action,21.143834,27.133276
Adventure,12.203745,14.291871
Animation,3.865879,3.515057
Childrens,8.672002,6.666487
Comedy,31.346647,29.310995
Crime,6.970239,8.432096
Documentary,0.726552,0.769003
Drama,42.769446,38.904003
Fantasy,1.410366,1.331951
Film-Noir,1.495843,1.815439


## Comments

1) I already have some questions. Number one, why is the timestamp this big number and can we get it to be a real timestamp?

2) What is the max rating possible for a movie (and min)?

3) Which movie has the most ratings?

4) Which genre has the highest ratings?

5) Can we convert these binary columns in "movies" to one column so we can do group by statements?

### Question 1

To many of you, storing the timestamp as a number (the number actually represents seconds) may appear odd. It is something called Unix time and is a computer science concept. The functions you have learned so far do not apply here, so we'll be leveraging a function from pandas called "to_datetime" that will allow us to do what we want.

In [159]:
# Let's try turning the timestamp from ratings
pd.to_datetime(ratings['timestamp'], unit='s')

0       1997-12-04 15:55:49
1       1998-04-04 19:22:22
2       1997-11-07 07:18:36
3       1997-11-27 05:02:03
4       1998-02-02 05:33:16
                ...        
99995   1997-11-22 05:10:44
99996   1997-11-17 19:39:03
99997   1997-09-20 22:49:55
99998   1997-12-17 22:52:36
99999   1997-11-19 17:13:03
Name: timestamp, Length: 100000, dtype: datetime64[ns]

In [160]:
# That looks about right so let's correct it
ratings['timestamp'] = pd.to_datetime(ratings['timestamp'], unit='s')
ratings.timestamp

0       1997-12-04 15:55:49
1       1998-04-04 19:22:22
2       1997-11-07 07:18:36
3       1997-11-27 05:02:03
4       1998-02-02 05:33:16
                ...        
99995   1997-11-22 05:10:44
99996   1997-11-17 19:39:03
99997   1997-09-20 22:49:55
99998   1997-12-17 22:52:36
99999   1997-11-19 17:13:03
Name: timestamp, Length: 100000, dtype: datetime64[ns]

In [161]:
# Let's also try to pull out just the date, year, month, day of week
ratings["date"] = ratings.timestamp.apply(lambda x: x.date())
ratings["year"] = ratings.timestamp.apply(lambda x: x.year)
ratings["month"] = ratings.timestamp.apply(lambda x: x.month)
ratings["dayofweek"] = ratings.timestamp.apply(lambda x: x.dayofweek)
ratings.head()

,user id,movie id,rating,timestamp,date,year,month,dayofweek
0,196,242,3,1997-12-04 15:55:49,1997-12-04,1997,12,3
1,186,302,3,1998-04-04 19:22:22,1998-04-04,1998,4,5
2,22,377,1,1997-11-07 07:18:36,1997-11-07,1997,11,4
3,244,51,2,1997-11-27 05:02:03,1997-11-27,1997,11,3
4,166,346,1,1998-02-02 05:33:16,1998-02-02,1998,2,0


### Question 2

Finding the max possible rating should be easy.

In [162]:
ratings.rating.max()

5

In [163]:
ratings.rating.min()

1

### Question 3

We're going to modify question three to be which Action movie has the most ratings?

In [164]:
# Filter DataFrame to only include Action movies
action = movies[movies['Action'] == 1]
mv = action[['movie title']]

mv.head()

,movie title
movie id,
2,GoldenEye
4,Get Shorty
17,From Dusk Till Dawn
21,Muppet Treasure Island
22,Braveheart


In [165]:
# Merge the mv DataFrame on ratings
mv.merge(ratings, how='inner', on='movie id')\
    .groupby(['movie title']).count()['user id']\
    .sort_values(ascending=False)

movie title
Star Wars                                      583
Return of the Jedi                             507
Air Force One                                  431
Independence Day (ID4)                         429
Raiders of the Lost Ark                        420
                                              ... 
King of New York                                 1
Police Story 4: Project S (Chao ji ji hua)       1
Bird of Prey                                     1
Power 98                                         1
Lashou shentan                                   1
Name: user id, Length: 249, dtype: int64

### Question 4

For this one we're going to need the movie_ratings DataFrame again

In [166]:
movie_ratings.head()

,movie id,movie title,release date,Action,Adventure,Animation,Childrens,Comedy,Crime,Documentary,...,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,user id,rating,timestamp
0,1,Toy Story,1-Jan-95,0,0,1,1,1,0,0,...,0,0,0,0,0,0,0,308,4,887736532
1,1,Toy Story,1-Jan-95,0,0,1,1,1,0,0,...,0,0,0,0,0,0,0,287,5,875334088
2,1,Toy Story,1-Jan-95,0,0,1,1,1,0,0,...,0,0,0,0,0,0,0,148,4,877019411
3,1,Toy Story,1-Jan-95,0,0,1,1,1,0,0,...,0,0,0,0,0,0,0,280,4,891700426
4,1,Toy Story,1-Jan-95,0,0,1,1,1,0,0,...,0,0,0,0,0,0,0,66,3,883601324


In [167]:
# Solution in class was to multiply the ratings column against each *Genre* column, and then taking the mean
# over the columns of the DataFrame

# We tried this in class but it didn't work
# movie_ratings.loc[:,'Action':'Western'] * movie_ratings.rating
# But we can use built-in methods to multiply across the rows of the DataFrame

# Specifying axis=0 tells pandas to multiply across the rows of the DataFrame
genre_ratings = movie_ratings.loc[:, 'Action':'Western'].multiply(movie_ratings.rating, axis=0)
genre_ratings.head()

,Action,Adventure,Animation,Childrens,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,0,0,4,4,4,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,5,5,5,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,4,4,4,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,4,4,4,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,3,3,3,0,0,0,0,0,0,0,0,0,0,0,0,0


In [168]:
# Why does this give us the wrong output?
genre_ratings.mean()

Action         0.890649
Adventure      0.481888
Animation      0.128953
Childrens      0.240854
Comedy         1.012621
Crime          0.292609
Documentary    0.027843
Drama          1.471227
Fantasy        0.043474
Film-Noir      0.067967
Horror         0.174967
Musical        0.174467
Mystery        0.190839
Romance        0.704890
Sci-Fi         0.453325
Thriller       0.767567
War            0.358646
Western        0.066997
dtype: float64

In [169]:
# Why does this work?
genre_ratings.replace(0, np.nan).mean().sort_values(ascending=False)

Film-Noir      3.921523
War            3.815812
Drama          3.687379
Documentary    3.672823
Mystery        3.638132
Crime          3.632278
Romance        3.621705
Western        3.613269
Animation      3.576699
Sci-Fi         3.560723
Musical        3.521397
Thriller       3.509007
Adventure      3.503527
Action         3.480245
Comedy         3.394073
Childrens      3.353244
Horror         3.290389
Fantasy        3.215237
dtype: float64

### Question 5

Is there a way for us to get all genres together into one column?

In [171]:
# If we only had one category per movie we could do something simple like this
movies.loc[:,'Action':'Western'].idxmax(axis=1)

movie id
1       Animation
2          Action
3        Thriller
4          Action
5           Crime
          ...    
1678        Drama
1679      Romance
1680        Drama
1681       Comedy
1682        Drama
Length: 1680, dtype: object

In [189]:
df = movies.loc[:, 'Action':'Western'].stack()

# Create multi-level index Series
df.head(10)

movie id             
1         Action         0
          Adventure      0
          Animation      1
          Childrens      1
          Comedy         1
          Crime          0
          Documentary    0
          Drama          0
          Fantasy        0
          Film-Noir      0
dtype: int64

In [196]:
#pd.Series(pd.Categorical(df[df!=0].index.get_level_values(1)))
movie_genre = df[df!=0].reset_index()
movie_genre.head(10)

,movie id,level_1,0
0,1,Animation,1
1,1,Childrens,1
2,1,Comedy,1
3,2,Action,1
4,2,Adventure,1
5,2,Thriller,1
6,3,Thriller,1
7,4,Action,1
8,4,Comedy,1
9,4,Drama,1


In [197]:
# Clean up the dataframe so that the index is movie id, the 0 column is dropped, and rename level_1 to genre
movie_genre = movie_genre.set_index('movie id').drop(0, axis=1).rename({'level_1': 'genre'}, axis=1)
movie_genre.head(10)

,genre
movie id,
1,Animation
1,Childrens
1,Comedy
2,Action
2,Adventure
2,Thriller
3,Thriller
4,Action
4,Comedy


In [199]:
genre_columns = ['Action', 'Adventure', 'Animation',
       'Childrens', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy',
       'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi',
       'Thriller', 'War', 'Western']

movie_genre = movie_genre.merge(movies, how='inner', left_index=True, right_index=True)
movie_genre.head(10)

,genre,movie title,release date,Action,Adventure,Animation,Childrens,Comedy,Crime,Documentary,...,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,num_genres
movie id,,,,,,,,,,,,,,,,,,,,,
1,Animation,Toy Story,1-Jan-95,0,0,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,3
1,Childrens,Toy Story,1-Jan-95,0,0,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,3
1,Comedy,Toy Story,1-Jan-95,0,0,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,3
2,Action,GoldenEye,1-Jan-95,1,1,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,3
2,Adventure,GoldenEye,1-Jan-95,1,1,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,3
2,Thriller,GoldenEye,1-Jan-95,1,1,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,3
3,Thriller,Four Rooms,1-Jan-95,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,1
4,Action,Get Shorty,1-Jan-95,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,3
4,Comedy,Get Shorty,1-Jan-95,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,3


In [200]:
movie_genre = movie_genre.drop(genre_columns, axis=1)
movie_genre.head(10)

,genre,movie title,release date,num_genres
movie id,,,,
1,Animation,Toy Story,1-Jan-95,3
1,Childrens,Toy Story,1-Jan-95,3
1,Comedy,Toy Story,1-Jan-95,3
2,Action,GoldenEye,1-Jan-95,3
2,Adventure,GoldenEye,1-Jan-95,3
2,Thriller,GoldenEye,1-Jan-95,3
3,Thriller,Four Rooms,1-Jan-95,1
4,Action,Get Shorty,1-Jan-95,3
4,Comedy,Get Shorty,1-Jan-95,3


In [103]:
user_movies = df.groupby("user id")["movie title"].apply(set).reset_index()
user_movies.head()

,user id,movie title
0,1,"{Jean de Florette , Psycho , Citizen Ruth , Fr..."
1,2,"{My Best Friend's Wedding , Air Force One , Up..."
2,3,"{Air Force One , Murder at 1600 , Crash , U Tu..."
3,4,"{Air Force One , One Flew Over the Cuckoo's Ne..."
4,5,"{Psycho , Star Trek: Generations , From Dusk T..."


In [92]:
user_movies["num_movies"] = user_movies["movie title"].apply(len)

In [95]:
user_movies[user_movies.num_movies > 200]

,user id,movie title,num_movies
0,1,"{Jean de Florette , Psycho , Citizen Ruth , Fr...",270
5,6,"{Jean de Florette , Psycho , Adventures of Rob...",207
6,7,"{Adventures of Robin Hood, The , Daniel Defoe'...",401
12,13,"{Adventures of Robin Hood, The , From Dusk Til...",631
17,18,"{Jean de Florette , Psycho , Wings of Desire ,...",277
...,...,...,...
893,894,"{Jean de Florette , Wings of Desire , Citizen ...",243
895,896,"{GoldenEye , Client, The , Star Trek: The Wrat...",362
915,916,"{From Dusk Till Dawn , GoldenEye , Immortal Be...",316
918,919,"{Citizen Ruth , Dream With the Fishes , Rainma...",215


In [96]:
user_movies.shape

(943, 3)

In [117]:
genre_rating = movie_ratings.loc[:,'Action':'Western'].multiply(movie_ratings.rating, axis=0)

In [118]:
genre_rating.head(5)

,Action,Adventure,Animation,Childrens,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,0,0,4,4,4,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,5,5,5,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,4,4,4,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,4,4,4,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,3,3,3,0,0,0,0,0,0,0,0,0,0,0,0,0


In [120]:
genre_rating.mean()

Action         0.890649
Adventure      0.481888
Animation      0.128953
Childrens      0.240854
Comedy         1.012621
Crime          0.292609
Documentary    0.027843
Drama          1.471227
Fantasy        0.043474
Film-Noir      0.067967
Horror         0.174967
Musical        0.174467
Mystery        0.190839
Romance        0.704890
Sci-Fi         0.453325
Thriller       0.767567
War            0.358646
Western        0.066997
dtype: float64

In [122]:
genre_rating.replace(0, np.nan).mean().sort_values(ascending=False)

Film-Noir      3.921523
War            3.815812
Drama          3.687379
Documentary    3.672823
Mystery        3.638132
Crime          3.632278
Romance        3.621705
Western        3.613269
Animation      3.576699
Sci-Fi         3.560723
Musical        3.521397
Thriller       3.509007
Adventure      3.503527
Action         3.480245
Comedy         3.394073
Childrens      3.353244
Horror         3.290389
Fantasy        3.215237
dtype: float64